# Distributions

### Preamble

### Distribution of missing values by company

### Distribution of missing values by feature

### Distribution of skew by feature

### Distribution of kurtosis by feature

### Distribution of interquartile ranges

### Distribution of intercorrelations

### Distribution of central tendency